# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Для начала импортируем необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from matplotlib import pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# загружаем классы для подготовки данных
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


# загружаем класс для работы с пропусками
from sklearn.impute import SimpleImputer

# загружаем функцию для работы с метриками
from sklearn.metrics import accuracy_score, f1_score


# импортируем класс RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV


# загружаем нужные модели
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
#Фиксируем константу для рандомизации
RANDOM_STATE = 42

# импортируем библиотеки для работы с текстом
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.shape

(159292, 3)

In [3]:
data.sample(5)

,Unnamed: 0,text,toxic
59234,59300,Sorry I have been unblocked already. It was no...,0
82080,82156,"Some of the people, places or things you have ...",0
12313,12327,"""\nNothing wrong with that portrait, but she w...",0
148756,148912,"""\n\nTrolling by Bogdan and Constantzeanu\nBul...",0
85442,85523,Rumor\nUntil Nick himself confirms this rumor ...,0


In [4]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

В классах явный дисбаланс - это нужно будет учитывать при обучении и классификации

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
data.duplicated().sum()

0

Дубликатов и пропусков в данных нет. Перейдём к их предобработке и трансформации, лишний стоблец удалим

In [7]:
data.columns

Index(['Unnamed: 0', 'text', 'toxic'], dtype='object')

In [8]:
data.drop('Unnamed: 0',axis=1,inplace=True)
data.sample(5)

,text,toxic
107169,Since when do staffers have the right to have ...,0
35911,Thank you for agreeing with me and taking the ...,0
152502,"Justification to add new proof, using Bézout's...",0
46534,Thanks and a Suggestion \n\nThank you for your...,0
39513,"""\n\nThank you, CrohnieGal!\nI'm still learnin...",0


Для начала лемматизируем текст. Сначала избавимся от посторонних символос с помощью регулярных выражений

In [9]:
data['lemm_text'] = data['text'].apply(lambda x: re.sub(r'[^a-zA-Z]'," ",x))

In [10]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m s...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on im...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember...


Далее используем специальный инструмент для лемматизации текста на английском языке

In [11]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
data['lemm_text'] = data['lemm_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w.lower()) for w in x.split()]))

In [13]:
corpus = data['lemm_text']

Используем стоп-слова для улучшения качества признакого описания

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
stopwords = set(stopwords.words('english'))

In [16]:
count_tf_idf = TfidfVectorizer(stop_words = list(stopwords))

Поделим данные на тестовую и тренировочную выборку

In [17]:
X_train, X_test, y_train, y_test = train_test_split(corpus,data['toxic'],test_size=0.25,random_state=RANDOM_STATE,stratify=data['toxic'])

Создадим матрицу TF-IDF для более качесвенного признакового описания постов

In [18]:
tf_idf_train = count_tf_idf.fit_transform(X_train)

In [19]:
tf_idf_test = count_tf_idf.transform(X_test)

## Обучение

Создадим пайплайн для обучения нескольких моделей кросс-валидацией с подбором гиперпараметров. Модели классфикации: Логистическая регрессия, решающее дерево, метод ближайших соседей

In [20]:


pipe_final = Pipeline(
[
    ('models',LogisticRegression( random_state=RANDOM_STATE, 
            solver='liblinear',class_weight='balanced'))
])

param_distr = [
    {
    'models' :[LogisticRegression( random_state=RANDOM_STATE, 
            solver='liblinear',class_weight='balanced')],
    'models__penalty':['l1','l2'],
    'models__C':range(1,20,5)
        
    },

    {
      
    'models':[KNeighborsClassifier()],
    'models__n_neighbors':[10,20]
    },
    
    {
    'models': [DecisionTreeClassifier(random_state=RANDOM_STATE,class_weight='balanced')],
        'models__max_depth': [2,5]

    }
]


In [21]:
randomized_search = RandomizedSearchCV(
pipe_final,
param_distributions=param_distr,
scoring='f1',
random_state=RANDOM_STATE,
n_jobs=-1)

randomized_search.fit(tf_idf_train,y_train)
print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('models',
                 LogisticRegression(C=11, class_weight='balanced',
                                    random_state=42, solver='liblinear'))])
Метрика лучшей модели на тренировочной выборке: 0.7622460466847006


Лучшей моделью оказалась модель логистической регрессии. Проверим метрику f_score на тестовой выборке

In [22]:
y_pred = randomized_search.predict(tf_idf_test)
f1_score(y_pred,y_test)

0.7611420612813371

## Выводы

В процессе решения задачи по классификации комментариев на "токсичные" и "нетоксичные" мной были сделаны следующие шаги и выводы:
    
* Загружены и подготовлены данные
* Произведена лемматизация данных и трансформация в матрицу TF_IDF
* Обучены насеолько моделей классификации с разными гиперпараметрами и получена лучшая

По итогу окзалось, что лучшей моделью по предсказанию классов стала модель Логистической регрессии, с метрикой F1 на тестовой выборке в 0.76, что соответствует требованию заказчика

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны